In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

from tqdm import tqdm
from sklearn import datasets
import random

In [3]:
import mylibrary.datasets as datasets

In [4]:
device = torch.device("cuda:0")
# device = torch.device("cuda:1")
# device = torch.device("cpu")

In [5]:
mnist = datasets.FashionMNIST()
# mnist.download_mnist()
# mnist.save_mnist()
train_data, train_label_, test_data, test_label_ = mnist.load()

train_data = train_data / 255.
test_data = test_data / 255.

train_size = len(train_label_)

In [6]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)
test_label = torch.LongTensor(test_label_)

In [7]:
## converting data to pytorch format
train_data = torch.Tensor(train_data)
test_data = torch.Tensor(test_data)
train_label = torch.LongTensor(train_label_)

In [8]:
input_size = 784
output_size = 10

In [9]:
class MNIST_Dataset(data.Dataset):
    
    def __init__(self, data, label):
        self.data = data
        self.label = label
        
#         self.label = mask.type(torch.float32).reshape(-1,1)
        self._shuffle_data_()
        
    def __len__(self):
        return len(self.data)
    
    def _shuffle_data_(self):
        randidx = random.sample(range(len(self.data)), k=len(self.data))
        self.data = self.data[randidx]
        self.label = self.label[randidx]
    
    def __getitem__(self, idx):
        img, lbl = self.data[idx], self.label[idx]
        return img, lbl

In [10]:
train_dataset = MNIST_Dataset(train_data, train_label)
test_dataset = MNIST_Dataset(test_data, test_label)

In [11]:
learning_rate = 0.0003
batch_size = 50

In [12]:
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

## Making of matrix factorized layer

In [13]:
class PairLinear(nn.Module):
    
    def __init__(self, input_dim, output_dim=None, bias=True):
        super().__init__()
        self.input_dim = input_dim
        if output_dim is None:
            output_dim = input_dim
        self.output_dim = output_dim
        
        self.num_factors = int(np.ceil(np.log2(max(self.input_dim, self.output_dim))))
        self.dim = 2**self.num_factors
        self.pair_indices = [self.get_pair_indices(self.dim, i+1) for i in range(self.num_factors)]
#         self.pair_indices = [tuple(ind.t()) for ind in self.pair_indices]
        
        self.weights = []
        for _ in range(self.num_factors):
            w = nn.Parameter(torch.randn(self.dim*2))
            self.weights.append(w)
        self.weights = nn.ParameterList(self.weights)
        self.bias = nn.Parameter(torch.zeros(self.output_dim))
        
        
#         self.zero_mat = torch.zeros(dim, dim)
        self.W = torch.eye(self.dim)
    
        if self.input_dim < self.dim:
            self.add_dim = torch.randint(0, self.input_dim, (self.dim-self.input_dim, ))
        if self.output_dim < self.dim:
            self.select_dim = torch.LongTensor(np.random.permutation(self.dim)[:self.output_dim])
        
    def forward(self, x):
        W = self.W.to(x.device)
        for pi, w in zip(self.pair_indices, self.weights):
            zero_mat = torch.zeros(self.dim, self.dim, device=x.device)
            zero_mat[pi] = w
            W = W@zero_mat
            
        
        if self.input_dim < self.dim:
            x = torch.cat([x, x[:, self.add_dim]], dim=1)
        y = x@W
        if self.output_dim < self.dim:
            y = y[:, self.select_dim]
        if self.bias is not None:
            y = y + self.bias
        return y

    def get_pair_indices(self, dim, step=1):
        assert 2**int(np.log2(dim)) == dim , "The dim must be power of 2"
        assert isinstance(step, int), "Step must be integer"

        blocks = (2**step)
        range_ = dim//blocks
        adder_ = torch.arange(0, range_)*blocks

        pairs_ = torch.Tensor([0, blocks//2])
        repeat_ = torch.arange(0, blocks//2).reshape(-1,1)

        block_map = (pairs_+repeat_).reshape(-1)
        reorder_for_pair = (block_map+adder_.reshape(-1,1)).reshape(-1)
        indx = reorder_for_pair.type(torch.long)
        indx = indx.reshape(-1, 2)

        map_idx = []
        for idx in indx:
            map_idx.append((idx[0], idx[0]))
            map_idx.append((idx[0], idx[1]))        
            map_idx.append((idx[1], idx[0]))        
            map_idx.append((idx[1], idx[1]))        
        map_idx = torch.LongTensor(map_idx)
        return map_idx

In [14]:
pfL = PairFactorizedLinear(100, 1024)

In [15]:
pfL(torch.randn(2, 100))

tensor([[  6.2016,   0.7034,   5.9664,  ..., -18.7599, -10.4431,  -1.3575],
        [ -4.7022,  -2.3834,   1.0551,  ...,   5.7297,  17.6619,   5.0865]],
       grad_fn=<AddBackward0>)

In [16]:
param_count = sum([torch.numel(p) for p in pfL.parameters()])
param_count

21504

In [17]:
1024*1024/param_count

48.76190476190476

In [18]:
class FactorNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.la1 = PairFactorizedLinear(784, 1024, bias=False)
        self.bn1 = nn.BatchNorm1d(1024)
        self.la2 = PairFactorizedLinear(1024, bias=False)
        self.bn2 = nn.BatchNorm1d(1024)
        self.la3 = nn.Linear(1024, 10)
        
    def forward(self, x):
        x = self.bn1(self.la1(x))
        x = torch.relu(x)
        x = self.bn2(self.la2(x))
        x = torch.relu(x)
        x = self.la3(x)
        return x

In [19]:
class OrdinaryNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.la1 = nn.Linear(784, 1024, bias=False)
        self.bn1 = nn.BatchNorm1d(1024)
        self.la2 = nn.Linear(1024, 1024)
        self.bn2 = nn.BatchNorm1d(1024, bias=False)
        self.la3 = nn.Linear(1024, 10)
        
    def forward(self, x):
        x = self.bn1(self.la1(x))
        x = torch.relu(x)
        x = self.bn2(self.la2(x))
        x = torch.relu(x)
        x = self.la3(x)
        return x

In [20]:
model = FactorNet()
param_count = sum([torch.numel(p) for p in model.parameters()])
param_count

57354

In [21]:
model = OrdinaryNet()
param_count1 = sum([torch.numel(p) for p in model.parameters()])
param_count1, param_count1/param_count

(1867786, 32.56592391114831)

### Model Development

In [26]:
torch.manual_seed(0)
model = FactorNet().to(device)
model = OrdinaryNet().to(device)
model

OrdinaryNet(
  (la1): Linear(in_features=784, out_features=1024, bias=True)
  (bn1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (la2): Linear(in_features=1024, out_features=1024, bias=True)
  (bn2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (la3): Linear(in_features=1024, out_features=10, bias=True)
)

In [27]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [28]:
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  1867786


In [29]:
losses = []
train_accs = []
test_accs = []
EPOCHS = 20

for epoch in range(EPOCHS):
    
    train_acc = 0
    train_count = 0
    for xx, yy in tqdm(train_loader):
        xx, yy = xx.to(device), yy.to(device)

        yout = model(xx)
        loss = criterion(yout, yy)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(float(loss))

        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        train_acc += correct
        train_count += len(outputs)

    train_accs.append(float(train_acc)/train_count*100)
    train_acc = 0
    train_count = 0

    print(f'Epoch: {epoch},  Loss:{float(loss)}')
    test_count = 0
    test_acc = 0
    for xx, yy in tqdm(test_loader):
        xx, yy = xx.to(device), yy.to(device)
        with torch.no_grad():
            yout = model(xx)
        outputs = torch.argmax(yout, dim=1).data.cpu().numpy()
        correct = (outputs == yy.data.cpu().numpy()).astype(float).sum()
        test_acc += correct
        test_count += len(xx)
    test_accs.append(float(test_acc)/test_count*100)
    print(f'Train Acc:{train_accs[-1]:.2f}%, Test Acc:{test_accs[-1]:.2f}%')
    print()

### after each class index is finished training
print(f'\t-> Train Acc {max(train_accs)} ; Test Acc {max(test_accs)}')

  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 0,  Loss:0.4663251042366028


  0%|          | 0/1200 [00:00<?, ?it/s]

Train Acc:84.99%, Test Acc:86.40%



  0%|          | 0/200 [00:00<?, ?it/s]

...

Train Acc:97.94%, Test Acc:89.55%



  0%|          | 0/200 [00:00<?, ?it/s]

Epoch: 19,  Loss:0.027473196387290955


100%|██████████| 200/200 [00:00<00:00, 651.95it/s]

Train Acc:97.96%, Test Acc:89.13%

	-> Train Acc 97.95833333333334 ; Test Acc 89.61


In [ ]:
## stats: 20 epochs
### for factor-net -> 57354 -> Train Acc 88.83666666666666 ; Test Acc 86.7
### for ordinary-net -> 1867786 -> Train Acc 97.95833333333334 ; Test Acc 89.61